In [1]:
import numpy as np
from loadData import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import syft as sy
import copy
import matplotlib.pyplot as plt
from torch import nn, optim

In [2]:
"""""
Attributes:
    -denseOutputData
    -sparseOutputData
    -denseInputData
    -denseSampleIndex
    -sparseInputData
    -sparseSampleIndex
    -allSamples
    -folds
"""""
data = DataLoader(datasetName='static')

In [3]:
hook = sy.TorchHook(torch)
hook.local_worker.is_client_worker = False

server = hook.local_worker

In [4]:
data.denseOutputData = data.denseOutputData.astype('float64')

In [5]:
x = torch.tensor(data.denseInputData, dtype=torch.float32, requires_grad=True)
# i = torch.LongTensor([data.sparseOutputData.row,
#                           data.sparseOutputData.col])
# v = torch.FloatTensor(data.sparseOutputData.data)
# y = torch.sparse.FloatTensor(i, v, torch.Size(data.sparseOutputData.shape)).requires_grad_(True)
y = torch.tensor(data.denseOutputData, dtype=torch.float32, requires_grad=True)
in_dim  = x.size()[1]
out_dim = y.size()[1]
x.cuda()
y.cuda()

RuntimeError: CUDA out of memory. Tried to allocate 2.23 GiB (GPU 0; 2.00 GiB total capacity; 0 bytes already allocated; 1.42 GiB free; 0 bytes reserved in total by PyTorch)

In [ ]:
# print(data.sparseOutputData.shape)
print(y.size())
print(x.size())
#plt.scatter(np.arange(data.sparseOutputData.data.size), data.sparseOutputData.data)
#plt.show()

In [ ]:
nworkers = 10
workers = []
data_per_worker = int(x.size()[0] / nworkers)
# data_per_worker = 1
x_pointers = []
y_pointers = []

#itt definiálom a workereket
print("Creating workers...")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")
for i in range(nworkers):
    print("Worker id:", 'w'+str(i), "has data from:",(i) * data_per_worker, "to", (i+1) * data_per_worker)
      
    #tenzorok másolása workereknek
#     xi = x[i * data_per_worker: (i+1) * data_per_worker].clone().detach().requires_grad_(True).tag('input_data')
#     yi = y[i * data_per_worker: (i+1) * data_per_worker].clone().detach().requires_grad_(True).tag('output_data')
#     xi = torch.tensor(data.denseInputData[i * data_per_worker: (i+1) * data_per_worker], dtype=torch.float32, requires_grad=True).tag('input_data')
#     yi = torch.tensor(data.denseOutputData[i * data_per_worker: (i+1) * data_per_worker], dtype=torch.float32, requires_grad=True).tag('output_data')
    
    xi = x[i * data_per_worker: (i+1) * data_per_worker].clone().tag('input_data')
    yi = y[i * data_per_worker: (i+1) * data_per_worker].clone().tag('output_data')
#     workers.append(sy.VirtualWorker(hook, id='w'+str(i), data=(xi, yi)))
    workers.append(sy.VirtualWorker(hook, id='w'+str(i)))
    
    x_pointers.append(xi.send(workers[i]))
    y_pointers.append(yi.send(workers[i]))
    
#     x_pointers.append(workers[i].search('input_data')[0])
#     y_pointers.append(workers[i].search('output_data')[0])

Innen kell újra csak

In [ ]:
latent_dim = 1000
class TrunkNet(nn.Module):
    def __init__(self):
        super(TrunkNet, self).__init__()
        self.layer1 = nn.Linear(in_features=in_dim, out_features=latent_dim)
        self.weight = self.layer1.weight
        self.bias = self.layer1.bias

    def forward(self, x):
        return torch.sigmoid(self.layer1(x))

In [ ]:
class HeadNet(nn.Module):
    def __init__(self):
        super(HeadNet, self).__init__()
        self.layer1 = nn.Linear(in_features=latent_dim, out_features=out_dim)
        self.weight = self.layer1.weight
        self.bias = self.layer1.bias

    def forward(self, x):
        return self.layer1(x)

In [ ]:
learningRate = 0.1

#ezt még használom később
trunkNet = TrunkNet()
trunkNet.cuda()
# trunkNet.build(torch.zeros(in_dim))
#ezt már nem
headNetInit = HeadNet()
headNetInit.cuda()
# headNetInit.build(torch.zeros(latent_dim))

head_pointers = []
optimizer_pointers = []

print("Sending head networks")
for i in range(nworkers):
    print("Sending model:",i)
    head_pointers.append(headNetInit.copy().send(workers[i]))
print("Models sent")

In [ ]:
# @sy.func2plan(args_shape=[(data_per_worker, out_dim),(data_per_worker, out_dim)])
# def plan_MSE(preds, expected):
#     return ((preds - expected) **2).sum() / data_per_worker
# plan_MSE.is_built

In [ ]:
loss_history = []
def train(epochs, worker_iters, batch_size):
    for epoch in range(1, epochs+1):
        
        print("Epoch", epoch)
        print(" Sending models...")
        
        trunk_pointers = []
        optimizer_pointers = []
        head_optimizer_pointers = []
        for i in range(nworkers):
            trunk_pointers.append(trunkNet.copy().send(workers[i]))
            optimizer_pointers.append(optim.SGD([
                {'params': trunk_pointers[i].parameters()},
                {'params': head_pointers[i].parameters(), 'lr': 0.001}
            ],lr=learningRate))


        losses = []
        for wi in range(worker_iters):
            preds = []
            losses = [None] * nworkers

            print(" Worker iterations", wi)
            for i in range(nworkers):
                print("  Worker", i, end =" ")
                for b in range(int(data_per_worker/batch_size)):
                    optimizer_pointers[i].zero_grad()
                    preds.append(head_pointers[i](trunk_pointers[i](x_pointers[i][b*batch_size:(b+1)*batch_size])))
                    losses[i] = (((preds[i] - y_pointers[i][b*batch_size:(b+1)*batch_size]) **2).sum() /data_per_worker / int(data_per_worker/batch_size))
                    losses[i].backward(retain_graph=True)
                
                    optimizer_pointers[i].step()
                
                losses[i] = losses[i].get().data
                
            print("")
        
        print(" Sending back models...")
        for i in range(nworkers):
            trunk_pointers[i].move(secure_worker)
                
        print(" Averaging weights")
        with torch.no_grad():
            sumw = 0
            sumb = 0
            for i in range(nworkers):
                sumw += trunk_pointers[i].weight.data
                sumb += trunk_pointers[i].bias.data
                
            trunkNet.weight.set_((sumw / nworkers).get())
            trunkNet.bias.set_((sumb / nworkers).get())

        sumloss = 0
        for i in range(nworkers):
            sumloss += losses[i]
            
        loss_history.append((sumloss/nworkers).item)
        print("Average loss:", sumloss/nworkers)

In [ ]:
train(20,1,32)

In [ ]:
plt.plot(loss_history)